In [ ]:
import numpy as np
from pathlib import Path
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
keyFileNameCorr = '/content/drive/MyDrive/Morteza/LungProject/PatientFilenameOrganizaiton/Lung1-7.xlsx'
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from collections import Counter
from sklearn.datasets import make_classification
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from matplotlib import pyplot
from numpy import where
from statistics import mean
from statistics import stdev
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
import numpy as np
from sklearn.decomposition import MiniBatchSparsePCA
from imblearn.over_sampling import RandomOverSampler 
from random import sample
clinical_416_address = '/content/drive/MyDrive/Morteza/LungProject/PatientFilenameOrganizaiton/416_Prediction_Clinical_variables.xlsx'
df_416 = pd.read_excel(clinical_416_address, dtype=str)
df_416['SexGroup'].replace(['Male', 'Female'],[0, 1], inplace=True)
df_416['StageCat'].replace(['Stage1-2', 'Stage3_4'],[0, 1], inplace=True)
df_key_filename = pd.read_excel(keyFileNameCorr, dtype=str)


Mounted at /content/drive


In [ ]:
stage1_address = '/content/drive/MyDrive/Morteza/LungProject/Stage1/stage1_progression.xlsx'
df_stage1 = pd.read_excel(stage1_address, dtype=str)
# df_stage1['SexGroup'].replace(['Male', 'Female'],[0, 1], inplace=True)
# df_stage1['StageCat'].replace(['Stage1-2', 'Stage3_4'],[0, 1], inplace=True)
label_column = df_stage1['Progression.1.yes..0..no'].values
df_key_filename = pd.read_excel(keyFileNameCorr, dtype=str)
file_name_column = df_stage1['File.name'].values

In [ ]:
random_state_num = 22

In [ ]:
cell_freq = pd.read_csv('/content/drive/MyDrive/Morteza/LungProject/CellFrequencies/cell_frequencies.csv', dtype=str)
cellFrequencies = cell_freq.iloc[:,range(4,38,2)].values
cellFrequencies = np.asarray(cellFrequencies, dtype=np.float64, order='C')
allFileNames = cell_freq['File name']


In [ ]:
allFileNames

0       Pano 01_Row5-2_1_ROI 07_H03-2758 C5-2_7
1        Pano 01_R5-R3_1_ROI 07_H04-0309 C2-1_7
2       Pano 01_Row9-7_1_ROI 08_H08-0549 C5-1_8
3        Pano 01_R8-R5_1_ROI 04_H08-0636 A2-1_4
4      Pano 01_Row5-2_1_ROI 11_H08-0669 J5-2_11
                         ...                   
411     Pano 02_Row5-4_2_ROI 01_H11-3292 D7-3_1
412     Pano 01_Row8-6_1_ROI 05_H12-1125 F7-1_5
413       Pano 01_Row8_1_ROI 03_H12-3271 A2-1_3
414     Pano 01_Row2-3_1_ROI 02_H12-4532 G4-1_2
415    Pano 01_Row8-6_1_ROI 14_H12-4607 G5-1_14
Name: File name, Length: 416, dtype: object

In [ ]:
used = set()
# mylist = [u'nowplaying', u'PBS', u'PBS', u'nowplaying', u'job', u'debate', u'thenandnow']
unique = [x for x in label_column if x not in used and (used.add(x) or True)]
import math
unique = [x for x in unique if not (isinstance(x, float) and math.isnan(x))]

print(unique)
y = []
X = []

target_0 = unique[0]
target_1 = unique[1]

['1', '0']


In [ ]:
X = []
y = []
# print(len(unique))



def remove_end_spaces(string):
    return "".join(string.rstrip())
for i in range(len(file_name_column)):
  ind_num = [index for index, value in enumerate(allFileNames[:416]) if value == remove_end_spaces(file_name_column[i])]
  
  y_val = label_column[i]
  # print(ind_num)

  if len(unique)==2:
    if y_val==target_0 or y_val==target_1:    
      
      if len(X)==0 and len(ind_num)!=0:
        y.append(y_val)
        X = cellFrequencies[ind_num,:]
        
      elif len(ind_num)!=0:
        y.append(y_val)
        X = np.vstack([X, cellFrequencies[ind_num,:]])
  elif len(unique) > 2:
    if y_val==target_0 or y_val==target_1 or y_val==target_2 or y_val==target_3 or y_val==target_4:    
      
      if len(X)==0 and len(ind_num)!=0:
        y.append(y_val)
        X = cellFrequencies[ind_num,:]
      elif len(ind_num)!=0:
        y.append(y_val)
        X = np.vstack([X, cellFrequencies[ind_num,:]])


copyX = X
copyY = y

In [ ]:
X.shape

(286, 17)

In [ ]:
X = copyX
y = copyY
A = Counter(y)
if len(unique)==2:
  class_0 = A.get(target_0)
  class_1 = A.get(target_1)
  n0 = int(class_0/10)
  n1 = int(class_1/10)
  baseline_dist = max(class_0,class_1)/(max(class_0,class_1)+min(class_0,class_1))
  ratio = max(class_0,class_1)/min(class_0,class_1)
elif len(unique) > 2:
  ratio = 2
print('Original dataset shape %s' % Counter(y))
if ratio >=2:
  ros = RandomOverSampler(random_state=42)
  X, y = ros.fit_resample(X, y)
  print('Resampled dataset shape %s' % Counter(y))


yy = np.array(y)
import numpy as np
from sklearn.model_selection import KFold
kf = KFold(n_splits=5,random_state=10,shuffle=True)
kf.get_n_splits(X)

delete_dup = True
# print(kf)

for train_index, test_index in kf.split(X):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = yy[train_index], yy[test_index]
    toBeDeleted = []
    if delete_dup: 
      for i in range(X_test.shape[0]):
        query = X_test[i]
        j = 0
        query_found = 0
        while j < X_train.shape[0] and query_found==0:
          match = X_train[j]
          dist = np.linalg.norm(query-match)
          if dist == 0:
            query_found = 1
            toBeDeleted.append(i)
          j = j + 1

      if X_test.shape[0]-len(toBeDeleted) >= 5:
        X_test = np.delete(X_test,np.s_[toBeDeleted],axis=0)
        y_test = np.delete(y_test,np.s_[toBeDeleted],axis=0)
    transformer = MiniBatchSparsePCA(n_components=9, batch_size=500,random_state=0)
    transformer.fit(X_train)
    X_tr_em = transformer.transform(X_train)
    X_te_em = transformer.transform(X_test)
    kernel = 1.0 * RBF(1.0)
    clf = make_pipeline(StandardScaler(), SVC(gamma=1e1,C=1e1,kernel = kernel))
    clf.fit(X_tr_em, y_train)
    print('test score = '+str(clf.score(X_te_em,y_test)))
    
A = Counter(copyY)
if len(unique)==2:
  class_0 = A.get(target_0)
  class_1 = A.get(target_1)
  ratio = max(class_0,class_1)/(max(class_0,class_1)+min(class_0,class_1))
  print('baseline = '+str(ratio))


Original dataset shape Counter({'0': 244, '1': 42})
Resampled dataset shape Counter({'1': 244, '0': 244})
test score = 0.9285714285714286
test score = 0.9215686274509803
test score = 0.9069767441860465
test score = 0.86
test score = 0.7954545454545454
baseline = 0.8531468531468531
